Классификация с помощью RNN с типом задачи "многие к одному"

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds
from collections import Counter

In [2]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')

target = df.pop('sentiment') # целевая метка

ds_raw = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [3]:
tf.random.set_seed(42)

ds_raw = ds_raw.shuffle(50000, reshuffle_each_iteration=False)

ds_raw_test = ds_raw.take(25000)
ds_raw_train_valid = ds_raw.skip(25000)
ds_raw_train = ds_raw_train_valid.take(20000)
ds_raw_valid = ds_raw_train_valid.skip(20000)

In [4]:
# переводим слова в числа и считаем частоты
tokenizer = tfds.deprecated.text.Tokenizer()
token_counts = Counter()

for ex in ds_raw_train:
    tokens = tokenizer.tokenize(ex[0].numpy()[0]) # .numpy() нужен для доступа в данные из tf.data.Dataset
    token_counts.update(tokens)

print(len(token_counts))

87119


In [5]:
encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
ex_str = 'This is a sample sentence, containing several words.'
print(encoder.encode(ex_str))

[53, 63, 10, 18918, 3164, 3299, 3721, 1233]


In [6]:
# для слов из проверочных или испытательных данных, которые не встретились в обучающем наборе, используется число или 0, или q + 1, где q - количество уникальных слов в обучающем наборе(в данном случае 87119)

def encode(text_tensor, label):
    # функция-кодировшик для преобразования строки в числа
    text = text_tensor.numpy()[0].decode('utf-8')
    encoded_text = encoder.encode(text)
    return encoded_text, label

def encode_map_fn(text, label):
    # функция для применения encode к каждому элементу в tf.data.Dataset, аналог map
    return tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))


ds_train = ds_raw_train.map(encode_map_fn)
ds_valid = ds_raw_valid.map(encode_map_fn)
ds_test = ds_raw_test.map(encode_map_fn)

for ex in ds_train.shuffle(1000).take(5):
    print(ex[0].shape)

(321,)
(164,)
(141,)
(235,)
(294,)


In [7]:
# в общем случае RNN способны работать с образцами разной длины, но мы почему-то будем решать эту проблему вручную с помощью дополнений(нулями или q + 1) до максимального
# количества слов образца причём не во всём наборе, а в текущем пакете

train_data = ds_train.padded_batch(32, padded_shapes=([-1], []))
valid_data = ds_valid.padded_batch(32, padded_shapes=([-1], []))
test_data = ds_test.padded_batch(32, padded_shapes=([-1], []))

In [8]:
# далее можно закодировать слова унитарным способом, то есть теперь слово - вектор, где есть только одна 1 и все остальные 0, и имеющий размер равный количеству уникальных слов, но тогда возникнет проклятие размерности, к тому же векторы будут разреженными
# поэтому будем использовать вложения(embeddings): каждое слово будет сопоставлено с вектором, содержащим вещественные числа(как правило от -1 до 1), который при этом имеет фиксированный размер, а также матрица таких векторов может обучаться вместо с нейросетью(по сути это матрица признаков)
# матрица вложений в данном случае будет входным слоем RNN

In [9]:
embedding_dim = 20
vocab_size = 87121

bi_lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)), # для прохода в обе стороны; можно заменить на SimpleRNN, но это будет менее эффективно, так как он будет не так хорошо улавливать долгосрочные зависимости
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

bi_lstm_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
bi_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy']
)

history = bi_lstm_model.fit(train_data, validation_data=valid_data, epochs=10)

test_results = bi_lstm_model.evaluate(test_data)
print('Test Acc.: {:.2f}%'.format(test_results[1]*100))

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 98s 153ms/step - accuracy: 0.6632 - loss: 0.5866 - val_accuracy: 0.7948 - val_loss: 0.4497
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 90s 143ms/step - accuracy: 0.8639 - loss: 0.3310 - val_accuracy: 0.7940 - val_loss: 0.4962
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 93s 149ms/step - accuracy: 0.6580 - loss: 0.6444 - val_accuracy: 0.5010 - val_loss: 0.8649
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 93s 148ms/step - accuracy: 0.7999 - loss: 0.4242 - val_accuracy: 0.8340 - val_loss: 0.4255
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 90s 145ms/step - accuracy: 0.9373 - loss: 0.1769 - val_accuracy: 0.8418 - val_loss: 0.4531
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 90s 143ms/step - accuracy: 0.9654 - loss: 0.1107 - val_accuracy: 0.8352 - val_loss: 0.5669
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 89s 143ms/step - accuracy: 0.9713 - loss: 0.0883 - val_accuracy: 0.8730 - val_loss: 0.4521
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 89s 142ms/step - accuracy: 0.9630 - loss: 0

In [15]:
# как ранее говорилось, SimpleRNN не может так эффективно работать с долгосрочными зависимостями, как Bidirectional LSTM, отчасти потому, что отзывы содержат слишком много слов, но бывают ситуации, когда такую проблему можно решить, просто обрезав тексты
# например, в данном случае, можно сделать предположение о том, что в отзывах только последние 100 слов выражают большую часть отношения к фильму, и если отзыв длинее этого, то брать только последние 100 слов

def preprocess_datasets(ds_raw_train, ds_raw_valid, ds_raw_test, max_length=None, batch_size=32):
    tokenizer = tfds.deprecated.text.Tokenizer()
    token_counts = Counter()

    for example in ds_raw_train:
        tokens = tokenizer.tokenize(example[0].numpy()[0])
        if max_length is not None:
            tokens = tokens[-max_length:]
        token_counts.update(tokens)

    print('Vocab-size:', len(token_counts))

    encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
    def encode(text_tensor, label):
        text = text_tensor.numpy()[0]
        encoded_text = encoder.encode(text)
        if max_length is not None:
            encoded_text = encoded_text[-max_length:]
        return encoded_text, label

    def encode_map_fn(text, label):
        return tf.py_function(encode, inp=[text, label], 
                              Tout=(tf.int64, tf.int64))

    ds_train = ds_raw_train.map(encode_map_fn)
    ds_valid = ds_raw_valid.map(encode_map_fn)
    ds_test = ds_raw_test.map(encode_map_fn)

    train_data = ds_train.padded_batch(
        batch_size, padded_shapes=([-1],[]))

    valid_data = ds_valid.padded_batch(
        batch_size, padded_shapes=([-1],[]))

    test_data = ds_test.padded_batch(
        batch_size, padded_shapes=([-1],[]))

    return (train_data, valid_data, 
            test_data, len(token_counts))

In [20]:
def build_rnn_model(embedding_dim, vocab_size,
                    recurrent_type='SimpleRNN',
                    n_recurrent_units=64,
                    n_recurrent_layers=1,
                    bidirectional=True):

    tf.random.set_seed(1)

    model = tf.keras.Sequential()
    
    model.add(
        tf.keras.layers.Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
            name='embed-layer')
    )
    
    for i in range(n_recurrent_layers):
        return_sequences = (i < n_recurrent_layers-1)
            
        if recurrent_type == 'SimpleRNN':
            recurrent_layer = tf.keras.layers.SimpleRNN(
                units=n_recurrent_units, 
                return_sequences=return_sequences,
                name='simprnn-layer-{}'.format(i))
        elif recurrent_type == 'LSTM':
            recurrent_layer = tf.keras.layers.LSTM(
                units=n_recurrent_units, 
                return_sequences=return_sequences,
                name='lstm-layer-{}'.format(i))
        elif recurrent_type == 'GRU':
            recurrent_layer = tf.keras.layers.GRU(
                units=n_recurrent_units, 
                return_sequences=return_sequences,
                name='gru-layer-{}'.format(i))
        
        if bidirectional:
            recurrent_layer = tf.keras.layers.Bidirectional(
                recurrent_layer, name='bidir-'+recurrent_layer.name)
            
        model.add(recurrent_layer)

    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    return model

In [21]:
batch_size = 32
embedding_dim = 20
max_seq_length = 100

train_data, valid_data, test_data, n = preprocess_datasets(
    ds_raw_train, ds_raw_valid, ds_raw_test, 
    max_length=max_seq_length, 
    batch_size=batch_size
)

vocab_size = n + 2

rnn_model = build_rnn_model(
    embedding_dim, vocab_size,
    recurrent_type='SimpleRNN', 
    n_recurrent_units=64,
    n_recurrent_layers=1,
    bidirectional=True)

rnn_model.summary()

rnn_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  metrics=['accuracy'])

history = rnn_model.fit(
    train_data, 
    validation_data=valid_data, 
    epochs=10)

results = rnn_model.evaluate(test_data)
print('Test Acc.: {:.2f}%'.format(results[1]*100))

Vocab-size: 57972


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embed-layer (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidir-simprnn-layer-0           │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.5068 - loss: 0.7020 - val_accuracy: 0.4988 - val_loss: 0.7076
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.5172 - loss: 0.6992 - val_accuracy: 0.7158 - val_loss: 0.5922
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.7273 - loss: 0.5657 - val_accuracy: 0.7404 - val_loss: 0.5942
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.7991 - loss: 0.4725 - val_accuracy: 0.7556 - val_loss: 0.5731
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.8126 - loss: 0.4355 - val_accuracy: 0.7226 - val_loss: 0.5699
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.8406 - loss: 0.3665 - val_accuracy: 0.7510 - val_loss: 0.5837
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.8456 - loss: 0.3259 - val_accuracy: 0.7488 - val_loss: 0.6960
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.8867 - loss: 0.2467 - 